In [6]:
import os
import platform

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = '4,3'
data_dir = './data'
output_dir = './output/novel_cnn'

print(platform.node())
print('data_dir: ', data_dir)
print('output_dir: ', output_dir)

bld
data_dir:  ./data
output_dir:  ./output/novel_cnn


In [7]:
from utils import prepare_data, seed_everything

train_dataset, val_dataset, test_dataset, _ = prepare_data(data_dir)
y_batch = next(iter(train_dataset))[0]

In [8]:
from models import NovelCNN

seed_everything(0)

model = NovelCNN()
model(y_batch)  # build the model by passing a sample input
model.summary(expand_nested=True)

Model: "novel_cnn_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block0 (Sequential)         (256, 256, 32)            3232      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| conv1 (Conv1D)             (256, 512, 32)            128      |
|                                                               |
| conv2 (Conv1D)             (256, 512, 32)            3104     |
|                                                               |
| pool (AveragePooling1D)    (256, 256, 32)            0        |
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯
 block1 (Sequential)         (256, 128, 64)            18560     
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| conv1 (Conv1D)             (256, 256, 64)            6208     |
|                                                               |
| conv2 (Conv1D)             (256, 256, 64)            

In [9]:
from keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, CSVLogger

from metrics import CC, RRMSE, SNR

model.compile(
    optimizer=Adam(learning_rate=5e-5, epsilon=1e-8),
    loss=MeanSquaredError(),
    metrics=[CC(), RRMSE(), SNR()],
)

callbacks = [
    EarlyStopping(
        monitor='val_snr',
        mode='max',
        patience=20,
        restore_best_weights=True,
        verbose=1,
    ),
    ModelCheckpoint(
        filepath=f'{output_dir}/novel_cnn.h5',
        mode='max',
        monitor='val_snr',
        save_best_only=True,
        save_weights_only=True,
    ),
    TensorBoard(log_dir=output_dir),
    CSVLogger(f'{output_dir}/novel_cnn.csv'),
]

history = model.fit(
    train_dataset,
    epochs=200,
    steps_per_epoch=200,
    validation_data=val_dataset,
    callbacks=callbacks,
)

Epoch 1/200
200/200 [==============================] - 12s 36ms/step - loss: 0.5963 - cc: 0.0625 - rrmse: 0.9983 - snr: 0.0150 - val_loss: 0.5040 - val_cc: 0.0810 - val_rrmse: 0.9978 - val_snr: 0.0190
Epoch 2/200
200/200 [==============================] - 7s 35ms/step - loss: 0.4927 - cc: 0.2274 - rrmse: 0.9747 - snr: 0.2297 - val_loss: 0.4678 - val_cc: 0.2704 - val_rrmse: 0.9611 - val_snr: 0.3523
Epoch 3/200
200/200 [==============================] - 7s 35ms/step - loss: 0.4579 - cc: 0.3146 - rrmse: 0.9508 - snr: 0.4499 - val_loss: 0.4415 - val_cc: 0.3633 - val_rrmse: 0.9326 - val_snr: 0.6168
Epoch 4/200
200/200 [==============================] - 7s 34ms/step - loss: 0.4392 - cc: 0.3752 - rrmse: 0.9280 - snr: 0.6639 - val_loss: 0.4250 - val_cc: 0.4019 - val_rrmse: 0.9156 - val_snr: 0.7813
Epoch 5/200
200/200 [==============================] - 7s 34ms/step - loss: 0.4258 - cc: 0.3910 - rrmse: 0.9210 - snr: 0.7330 - val_loss: 0.4110 - val_cc: 0.4291 - val_rrmse: 0.9015 - val_snr: 0.9206

In [10]:
model.evaluate(test_dataset)

22/22 [==============================] - 0s 10ms/step - loss: 0.0821 - cc: 0.9200 - rrmse: 0.3505 - snr: 10.5157


[0.08205802738666534,
 0.9199916124343872,
 0.35051077604293823,
 10.515700340270996]